# Import Necessary Libraries

In [ ]:
import pandas as pd
from random import sample
import plotly.express as px
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)

# Data Analysis

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("testFeatures.csv")
sample_sub_df = pd.read_csv("sample_solution.csv")

There is no null value.

In [ ]:
train_df.isnull().sum().sum()

The date is between 2016 and 2020.

In [ ]:
train_df["tarih"].unique()

There are 79 different ürün.

In [ ]:
train_df["ürün"].nunique()

In [ ]:
train_df["ürün"].unique()

In [ ]:
print(train_df["tarih"].nunique())

print(train_df["ürün"].nunique())

print(train_df.groupby("ürün")["ürün besin değeri"].nunique().max())

print(train_df.groupby("ürün")["ürün kategorisi"].nunique().max())

print(train_df.groupby("ürün")["ürün üretim yeri"].nunique().max())

print(train_df.groupby("ürün")["market"].nunique().max())

print(train_df.groupby("ürün")["şehir"].nunique().max())

There are all combinations in the dataset.

In [ ]:
60*79*2*3*8

In [ ]:
(60*79*2*3*8)==len(train_df)

Test dataset combinations are the same with the train dataset because the amount of data per year is equal.

In [ ]:
(227520/5)==len(test_df)

In [ ]:
print(test_df["tarih"].nunique())

print(test_df["ürün"].nunique())

print(test_df.groupby("ürün")["ürün besin değeri"].nunique().max())

print(test_df.groupby("ürün")["ürün kategorisi"].nunique().max())

print(test_df.groupby("ürün")["ürün üretim yeri"].nunique().max())

print(test_df.groupby("ürün")["market"].nunique().max())

print(test_df.groupby("ürün")["şehir"].nunique().max())

In [ ]:
test_df["tarih"].unique()

Plot to examine the behavior.

In [ ]:
unique_urun = list(train_df["ürün"].unique())
unique_urun_uretim_yeri = list(train_df["ürün üretim yeri"].unique())
unique_market = list(train_df["market"].unique())
unique_sehir = list(train_df["şehir"].unique())

In [ ]:
sample_urun = sample(unique_urun, 1)[0]
sample_urun_uretim_yeri = sample(unique_urun_uretim_yeri, 1)[0]
sample_market = sample(unique_market, 1)[0]
sample_sehir = sample(unique_sehir, 1)[0]
sample_df = train_df[(train_df["ürün"]==sample_urun)&(train_df["ürün üretim yeri"]==sample_urun_uretim_yeri)&(train_df["market"]==sample_market)&(train_df["şehir"]==sample_sehir)]

print("ürün:", sample_urun)
print("ürün üretim yeri:", sample_urun_uretim_yeri)
print("market:", sample_market)
print("sehir:", sample_sehir)

fig = px.line(sample_df, x="tarih", y="ürün fiyatı", title='Price of product over time')
fig.show()

Most of the plots seem easy to predict. A basic timeseries model should be enough to capture the patterns in the data.

Price is heavily affected by the type of the ürün as expected.

In [ ]:
train_df.groupby("ürün")["ürün fiyatı"].mean()

ürün üretim yeri does not seem very important for the ürün fiyatı.

In [ ]:
train_df.groupby("ürün üretim yeri")["ürün fiyatı"].mean()

market does not seem very important for the ürün fiyatı.

In [ ]:
train_df.groupby("market")["ürün fiyatı"].mean()

şehir does not seem very important for the ürün fiyatı.

In [ ]:
train_df.groupby("şehir")["ürün fiyatı"].mean()

# Solution Discussion

We can approach the problem in several ways. It can be seen as a machine learning problem or a time series problem. The former type of problems are solved with ML algorithms but for the second ones simpler, statistical based methods are used. 

We have totally 79\*2\*3\*8 = 3792 dimensions and for each dimension, there are 60 values in the train data. So, there is a tradeoff here. If we use each dimension separately, we better represent the changes in that dimension however it is more likely to overfit. If we use all the data to train the model, it is less likely to overfit but we may miss the ürün based patterns. So, the best option seems to train a simple, generalizable model for each dimension. Also, from the plots, we see that it is easy to solve time series problem. In this way, we will train 3792 models differently and use each of them for each dimension. Then, the total prediction will be combined. 

The real difficulty of the task is the change in price dynamics in the year 2021. We can teach the patterns in the past between 2016 and 2020 to the models however they can not learn to predict the year 2021. Because any model learning from data, it could be a machine learning model or a statistical model, only represents its training data and produces output accordingly. So, the outputs of our model will be probably lower than the expected value. This is obvious however we have to put it into a mathematical model to optimize for the target objective, RMSE in this case.

So, we should be able to predict the next year according to the pattern in the train data and add some additional predictive factors about 2021 to get good results. 

For intuition, let's examine some basics of time series analysis. There is a technique called time series decomposition, for additional information https://otexts.com/fpp2/components.html, which splits the data over time into components such as trend, seasonality and remaining. Generally, time series models learn to capture the trend and seasonality in a disentangled manner and use them to predict the future. The remaining part can be seen as the noise in the data and it is unpredictable by its nature. 

One strong example model using this intuition is Prophet model of Facebook, https://facebook.github.io/prophet/. It is an additive regression model, https://research.facebook.com/blog/2017/02/prophet-forecasting-at-scale/, which works similar to what I told in the previous paragraph. It combines four components additively and produces its predictions. Its components are a piecewise linear or logistic growth curve trend, a yearly seasonal component modeled using Fourier series, a weekly seasonal component using dummy variables and a user-provided list of important holidays. The model tries to find the necessary parameters of its mathematical functions from the training data in a disentangled way. So, one can see the trend or seasonality components of predictions.

We use this basic time series decomposition logic in our problem. We will teach our model to capture the trend and seasonality in our training data. Then, the model will be able to predict the trend and the seasonality of the year 2021. However, we need some additional additive components. In addition to trend and seasonality, we should find some more components special to the year 2021. To gain insights about what kind of components they could be, we should look for external data. So, I used external data to find the types of components I will use and then, I will tune the parameters of components with the data for 2021.

I did not use the currency data directly even if it seems highly correlated with the target. There are a few reasons behind that. There are increases and decreases in the dollar-tl graph however the price of a product does not decrease even if the value of the dollar against tl decreases. Also, the effect is lagged and the lag time may change from time to time. So, it may make the predictions stronger but it also increases the complexity of the problem. That's why I didn't prefer to integrate it directly into the models. 

When we look at the data, https://data.tuik.gov.tr/Bulten/Index?p=Tuketici-Fiyat-Endeksi-Aralik-2021-45789, we see that overall inflation in 2021 is high compared to previous years. Also, the inflation rate increases over time. Moreover, we see a sharp increase in december. So, there are 3 insights that can be used as components. There should be a component representing the increase in all year, another component representing the increase in inflation rate over time, and another one representing the difference in december. So, we have totally 5 additive components which will be used in the final prediction.

We used external data to find the types of components. Trend and seasonality components can be predicted easily from the training data and we will use the year 2020 as a validation set to improve the predictions of our model about trend and seasonality. However, we have to tune additional 3 parameters of our model. The only way to predict their value is again external data and also we can tune the results according to the feedback coming from Kaggle. We have very limited submissions so we will find a technique to use them efficiently. We only have 18 different submissions and values of 3 parameters should be tuned. We should not use our submissions for trend and seasonality components but we should trust our validation results.

Using external data and Kaggle scores, a point near to optimum point can be found in a few submissions. The final parameters are %5 overall increase, %0-35 increase starting from %0 in January and finishing with %35 in December and %0 additional increase in December in addition to trend and seasonality components.

In a real life case, the parameters of components can be guessed by using insights and predictions can be produced.

The component structure is important because we interpret predictions of different components and the shape of components should be compatible with the real values. If we try to model a decreasing inflation rate, a constant increase in inflation rate or any other possibility, we may not be able to get low errors. The components should represent the test data. When we look at the final score, it seems to represent well.

# Trend and Seasonality Validation

We will try different models to predict trend and seasonality components. A good model can be created using the year 2020 as the validation set and the rest as the training set. That model will be able to generalize the year 2021.

Separate data into train and validation set.

In [ ]:
val_df = train_df[train_df["tarih"].str.contains("2020")].reset_index(drop=True)
train_val_df = train_df[~train_df["tarih"].str.contains("2020")]
train_df_sorted = train_df.sort_values(by=["ürün", "ürün üretim yeri", "market", "şehir", "tarih"]).reset_index(drop=True)

val_df_sorted = val_df.sort_values(by=["ürün", "ürün üretim yeri", "market", "şehir", "tarih"]).reset_index(drop=True)
train_val_df_sorted = train_val_df.sort_values(by=["ürün", "ürün üretim yeri", "market", "şehir", "tarih"]).reset_index(drop=True)

Calculate some dictionaries to use in next steps.

In [ ]:
ürün_to_max_fiyat_dict = dict(train_val_df.groupby("ürün")["ürün fiyatı"].max())
ürün_to_max_fiyat_dict_14 = {temp_key:ürün_to_max_fiyat_dict[temp_key]*1.14 for temp_key in ürün_to_max_fiyat_dict}

ürün_dimensions_to_max_fiyat_dict = dict(train_val_df.groupby(["ürün", "ürün üretim yeri", "market", "şehir"])["ürün fiyatı"].max())
ürün_dimensions_to_max_fiyat_dict_14 = {temp_key:ürün_dimensions_to_max_fiyat_dict[temp_key]*1.14 for temp_key in ürün_dimensions_to_max_fiyat_dict}

Max price for each ürün

In [ ]:
print("rmse:", mean_squared_error(val_df["ürün fiyatı"], val_df["ürün"].map(ürün_to_max_fiyat_dict), squared=False))
print("r2:", r2_score(val_df["ürün fiyatı"], val_df["ürün"].map(ürün_to_max_fiyat_dict)))

Increased max price for each ürün

In [ ]:
print("rmse:", mean_squared_error(val_df["ürün fiyatı"], val_df["ürün"].map(ürün_to_max_fiyat_dict_14), squared=False))
print("r2:", r2_score(val_df["ürün fiyatı"], val_df["ürün"].map(ürün_to_max_fiyat_dict_14)))

Max price for each combination

In [ ]:
print("rmse:", mean_squared_error(val_df["ürün fiyatı"], [ürün_dimensions_to_max_fiyat_dict[each] for each in list(zip(val_df["ürün"], val_df["ürün üretim yeri"], val_df["market"], val_df["şehir"]))], squared=False))
print("r2:", r2_score(val_df["ürün fiyatı"], [ürün_dimensions_to_max_fiyat_dict[each] for each in list(zip(val_df["ürün"], val_df["ürün üretim yeri"], val_df["market"], val_df["şehir"]))]))

Increased max price for each combination

In [ ]:
print("rmse:", mean_squared_error(val_df["ürün fiyatı"], [ürün_dimensions_to_max_fiyat_dict_14[each] for each in list(zip(val_df["ürün"], val_df["ürün üretim yeri"], val_df["market"], val_df["şehir"]))], squared=False))
print("r2:", r2_score(val_df["ürün fiyatı"], [ürün_dimensions_to_max_fiyat_dict_14[each] for each in list(zip(val_df["ürün"], val_df["ürün üretim yeri"], val_df["market"], val_df["şehir"]))]))

ARIMA model

In [ ]:
results = []
for i in tqdm.tqdm(range(int(len(train_val_df_sorted)/48))):
    temp_df = train_val_df_sorted[i*48:(i+1)*48]
    
    prophet_df = temp_df[["tarih", "ürün fiyatı"]]
    prophet_df.columns = ["ds", "y"]

    # m = Prophet(weekly_seasonality=False, yearly_seasonality=False, daily_seasonality=False)
    model = ARIMA(list(prophet_df["y"]), order=(3,1,0))
    model_fit = model.fit()

    results += list(model_fit.forecast(12))
    
print("rmse:", mean_squared_error(val_df_sorted["ürün fiyatı"], results, squared=False))
print("r2:", r2_score(val_df_sorted["ürün fiyatı"], results))

Prophet model

In [ ]:
results = []
for i in tqdm.tqdm(range(int(len(train_val_df_sorted)/48))):
    temp_df = train_val_df_sorted[i*48:(i+1)*48]
    
    prophet_df = temp_df[["tarih", "ürün fiyatı"]]
    prophet_df.columns = ["ds", "y"]

    # m = Prophet(weekly_seasonality=False, yearly_seasonality=False, daily_seasonality=False)
    m = Prophet(weekly_seasonality=False, daily_seasonality=False)
    m.fit(prophet_df)

    future = m.make_future_dataframe(periods=12 , freq='M')

    forecast = m.predict(future)

    results += list(forecast["yhat"][-12:])

print("rmse:", mean_squared_error(val_df_sorted["ürün fiyatı"], results, squared=False))
print("r2:", r2_score(val_df_sorted["ürün fiyatı"], results))

Prophet model has low error and quite high r2 score. So, we can continue with it. More model trials such as machine learning and deep learning techniques can be applied however we already have a quite good model and the difficult part of the competition is to capture the differences in year 2021. So, I focused to predict the year 2021 better.

# Solution

After finding the correct model for trend and seasonality components and proposing three new components for year 2021, the code in the rest is pretty easy.

Sort values by each dimension.

In [ ]:
train_df_sorted = train_df.sort_values(by=["ürün", "ürün üretim yeri", "market", "şehir", "tarih"]).reset_index(drop=True)
test_df_sorted = test_df.sort_values(by=["ürün", "ürün üretim yeri", "market", "şehir", "tarih"]).reset_index(drop=True).copy()

Get Prophet predictions for each dimension.

In [ ]:
results = []
for i in tqdm.tqdm(range(int(len(train_df_sorted)/60))):
    temp_df = train_df_sorted[i*60:(i+1)*60]
    
    prophet_df = temp_df[["tarih", "ürün fiyatı"]]
    prophet_df.columns = ["ds", "y"]

    m = Prophet(weekly_seasonality=False, daily_seasonality=False)
    m.fit(prophet_df)

    future = m.make_future_dataframe(periods=12 , freq='M')

    forecast = m.predict(future)

    results += list(forecast["yhat"][-12:])
test_df_sorted["ürün fiyatı"] = results

First two componenets can be added together. They are overall high inflation rate and increasing inflation rate over time.

In [ ]:
coef = np.arange(1.05, 1.40, 0.030)
for i in range(12):
    test_df_sorted["ürün fiyatı"].loc[test_df_sorted["tarih"].str.contains("-{}-".format("0"*(2-len(str(i+1)))+(str(i+1))))] = test_df_sorted[test_df_sorted["tarih"].str.contains("-{}-".format("0"*(2-len(str(i+1)))+(str(i+1))))]["ürün fiyatı"]*coef[i]

Component related to december is added. It does not have a value according to the hyperparameter optimization so december inflation rate is not very high compared to other months in our test data set.

In [ ]:
test_df_sorted["ürün fiyatı"].loc[test_df_sorted["tarih"].str.contains("-12-")] = test_df_sorted[test_df_sorted["tarih"].str.contains("-12-")]["ürün fiyatı"]*1.00

Write results to a csv file.

In [ ]:
test_df_sorted = test_df_sorted[["id", "ürün fiyatı"]]

test_df_sorted = test_df_sorted.sort_values(by="id").reset_index(drop=True)

test_df_sorted.to_csv("final_submission.csv", index=False)